## In this notebook we are trying to create query engines of three different kind and also a functional tool for the llamaindex agent to use the tools smartly choose what ever it finds the best for the use case.

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from google.oauth2 import service_account
from llama_index.llms.vertex import Vertex
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings

# get API key and create embeddings
from dotenv import load_dotenv
import os
load_dotenv()
goog_api_key = os.getenv("GOOGLE_API_KEY")


# Replace with the path to your service account key file
filename = "key.json"
credentials = service_account.Credentials.from_service_account_file(filename)


# Initialize the Vertex AI model
llm = Vertex(
    model="gemini-pro",  # Specify the model, e.g., "text-bison" or "gemini-pro"
    project=credentials.project_id,
    credentials=credentials,
    temperature=0.0,  # Adjust as needed
    additional_kwargs={}
)

# setting the embed model
model_name = "models/text-embedding-004"

embed_model_gemini = GeminiEmbedding(
    model_name=model_name, api_key=goog_api_key, title="this is a document"
)

# setting the embed model
Settings.embed_model = embed_model_gemini

In [4]:
# Define folders for indexing
folders = ["organic_farming", "government_schemes", "packing_products"]
query_engines = {}

for folder in folders:
    # Load data and create index
    documents = SimpleDirectoryReader(f"data/{folder}").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=f"storage/{folder}")
    
    # Create a query engine
    query_engines[folder] = index.as_query_engine()


In [5]:
query_engines

{'organic_farming': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x17cc90090>,
 'government_schemes': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x17eca9550>,
 'packing_products': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x17ed38890>}

In [6]:
# Query specific engines
query = "What are the current government schemes for organic farming?"
response = query_engines["government_schemes"].query(query)

print("Response from government schemes:", response.response)

Response from government schemes: The current government schemes for organic farming include the Paramparagat Krishi Vikas Yojana (PKVY), which is a central government initiative aimed at promoting organic farming through a cluster-based approach. This scheme supports the use of bio-fertilizers, compost, and other organic inputs to ensure long-term soil fertility and environmental conservation.


In [9]:
# Combine query response with weather info
region = "Rajasthan"
query = "What are the best crops to grow in this season?"
response = query_engines["organic_farming"].query(query)
print(response.response)

The best crops to grow in this season would be those that benefit from crop diversity, promote soil fertility, and have a wide range of essential nutrients. Additionally, crops that can be grown using organic farming practices such as green manure, composting, crop rotation, cover cropping, and reduced tillage would be ideal choices. Legumes like soybeans, corn, and rice, along with fruits like bananas and coffee, could be good options to consider for planting in this season.


In [10]:
# questions about packing products
query = "What are the best packing products for organic farming?"
response = query_engines["packing_products"].query(query)
print(response.response)

Biodegradable Packaging, Recycled Paper Packaging, Compostable Bags, Molded Pulp Packaging, Bamboo Packaging, Edible Packaging, and Compostable Films are some of the best packing products for organic farming.


## working with tools

In [17]:
from llama_index.core.tools import FunctionTool
import requests

# Tool to query vector indexes
def query_index(folder, query):
    storage_context = StorageContext.from_defaults(persist_dir=f"storage/{folder}")
    index = load_index_from_storage(storage_context)
    query_engine = index.as_query_engine()
    response = query_engine.query(query)
    return response.response

# Create FunctionTool instances for each index
index_tools = [
    FunctionTool.from_defaults(
        fn=lambda query, folder=folder: query_index(folder, query),
        name=f"query_{folder}_index",
        description=f"Query the {folder.replace('_', ' ')} index."
    )
    for folder in folders
]



## creating the weather api for invoving it with the functional tools 

In [ ]:
import requests
import os


# Get API key from .env file
API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")


# Base URL for OpenWeatherMap API
BASE_URL = os.getenv("OPENWEATHERMAP_BASE_URL")

def get_weather_data(city: str):
    """
    Fetches weather data for a given city using OpenWeatherMap API.
    
    Args:
        city (str): Name of the city to fetch weather for.
    
    Returns:
        dict: Weather data for the city or error message.
    """
    params = {
        'q': city,
        'appid': API_KEY,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }
    
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
        weather_data = response.json()
        return weather_data
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}


In [42]:
get_weather_data("Dehradun")

{'coord': {'lon': 78.0437, 'lat': 30.3256},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 11.19,
  'feels_like': 9.2,
  'temp_min': 11.19,
  'temp_max': 11.19,
  'pressure': 1018,
  'humidity': 32,
  'sea_level': 1018,
  'grnd_level': 921},
 'visibility': 10000,
 'wind': {'speed': 2.48, 'deg': 38, 'gust': 1.95},
 'clouds': {'all': 0},
 'dt': 1734182967,
 'sys': {'type': 1,
  'id': 9162,
  'country': 'IN',
  'sunrise': 1734140187,
  'sunset': 1734176927},
 'timezone': 19800,
 'id': 1273313,
 'name': 'Dehradun',
 'cod': 200}

In [20]:
from llama_index.core.tools import FunctionTool

# Define the weather tool
weather_tool = FunctionTool.from_defaults(
    fn=get_weather_data,
    name="get_weather",
    description="Fetch live weather data for a given city. Provide the city name as input."
)


In [21]:
# Combine all tools
tools = index_tools + [weather_tool]

In [ ]:
from llama_index.agent.openai import OpenAIAgent

from llama_index.llms.openai import OpenAI

...

# initialize llm
llm2 = OpenAI(model="gpt-4o-mini")

# Initialize the agent with tools and LLM
agent = OpenAIAgent.from_tools(tools=tools, llm=llm2, verbose=True)


In [29]:
query = "how to grow tomatoes in winter in dehradun?"

In [ ]:
response = agent.chat(query)

In [36]:
print(response.response)

It seems there is an issue with retrieving specific information about growing tomatoes in winter in Dehradun. However, I can provide you with general tips on how to grow tomatoes in winter:

1. **Choose the Right Variety**: Select cold-resistant tomato varieties that can thrive in lower temperatures.

2. **Use Greenhouses or Row Covers**: Protect your plants from frost by using a greenhouse or row covers. This will help maintain a warmer environment.

3. **Soil Preparation**: Ensure the soil is well-drained and rich in organic matter. You can add compost to improve soil fertility.

4. **Planting**: Start seeds indoors and transplant them outside when the weather is suitable. If planting directly, ensure the soil temperature is warm enough.

5. **Watering**: Water the plants regularly, but avoid overwatering. Ensure the soil remains moist but not soggy.

6. **Fertilization**: Use a balanced fertilizer to provide essential nutrients. Fertilize every few weeks during the growing season.



In [38]:
agent2 = OpenAIAgent.from_tools(
    tools=tools,
    llm=llm2,
    system_prompt=(
        "You are an expert assistant for farmers. Respond to queries accurately and "
        "format your answers according to the following guidelines when necessary: \n"
        "- For 'best practices' queries, list items with bullets.\n"
        "- For weather-related queries, provide data in a concise summary.\n"
        "- For government scheme queries, include eligibility, benefits, and how to apply."
    ),
    verbose=True

)


In [39]:
response = agent2.chat(query)

Added user message to memory: how to grow tomatoes in winter in dehradun?


In [40]:
response.response

'Growing tomatoes in winter, especially in a region like Dehradun, requires careful planning and management. Here are some best practices to follow:\n\n- **Choose the Right Variety**: Select cold-tolerant tomato varieties that can withstand lower temperatures.\n\n- **Use Greenhouses or Polyhouses**: Protect your plants from frost and maintain a warmer environment by growing them in greenhouses or polyhouses.\n\n- **Soil Preparation**: Ensure well-drained, nutrient-rich soil. Incorporate organic matter like compost to improve soil quality.\n\n- **Temperature Control**: Use heating systems or row covers to maintain optimal temperatures, especially during the night.\n\n- **Watering**: Water the plants adequately but avoid overwatering. Ensure proper drainage to prevent root rot.\n\n- **Fertilization**: Use balanced fertilizers to promote healthy growth. Consider using organic fertilizers for better results.\n\n- **Pest Management**: Monitor for pests and diseases, and use organic pesticid

## creating storage spaces for three different types of querstions and rag spaces

In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext, SimpleDirectoryReader

storage_contexts = {}

# Load data and create index for each folder

for folder in ['organic_farming', 'government_schemes', 'packing_products']:
    documents = SimpleDirectoryReader(f"data/{folder}").load_data()
    storage_context = StorageContext.from_defaults(persist_dir=f"storage/{folder}")
    index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)
    index.storage_context.persist()
    storage_contexts[folder] = storage_context


In [ ]:
from llama_index import load_index_from_storage

# Load the indexes from storage
def query_index(folder, query):
    storage_context = StorageContext.from_defaults(persist_dir=f"storage/{folder}")
    index = load_index_from_storage(storage_context)
    query_engine = index.as_query_engine()
    response = query_engine.query(query)
    return response.response

In [ ]:
import requests

# Get API key from .env file
def get_weather(region):
    api_key = "YOUR_OPENWEATHERMAP_API_KEY"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={region}&appid={api_key}"
    response = requests.get(url)
    return response.json()
